# Short-term memory task and n-bit parity check task

1. Overview
2. Purpose
3. Preparation
4. Implementation
  - Echo state network and classification setup
  - Short-term memory task
  - N-bit parity check task
5. Demonstration

# 1. Overview
Let's implement two tasks, **short-term memory task for binary input** and **N-bit parity check one**.
These tasks were introduced by [Nakajima et al., 2014](https://royalsocietypublishing.org/doi/full/10.1098/rsif.2014.0437) and used for assessing the degree of the physical system's information processing; that is, **memory and nonlinearity**. 
Suppose we have a $D$-dimensional descrete dynamical system $\boldsymbol{x}(t) \in \mathbb{R}^{D},~t\in\{\cdots -2, -1,0,1, 2 \cdots\}$ with a binary input sequence $s(t)\in\{0,~1\}$;
$$
\begin{align*}
\\
\def\bm#1{{\boldsymbol #1}}
\bm{x}(t+1) = f(\bm{x}(t),~s(t)).
\\
\\
\end{align*}
$$
The short-term memory task evaluates how successfully the system can restore the past sequence by using the following target function $d_{N}^{\text{memory}}(t)$ based on the current state $\bm{x}(t)$;
$$
\begin{align*}
\\
d_{N}^{\text{memory}}(t) = s(t-N).
\\
\\
\end{align*}
$$

On the other hand, the objective of the N-bit parity check task is to count and check if the number of "1" in the N-past binary sequence is **even or odd** based on the current state of the system $\bm{x}(t)$.
In other words, the target function $d_{N}^{\text{parity}}(t)$ can be defined as follows;
$$
\begin{align*}
\\
d_{N}^{\text{parity}}(t) &= \left( \sum_{t^{'}=1}^{N} s(t-t^{'})\right) \% 2 .
\\
\\
\end{align*}
$$
Since both the target functions take discrete output values of $\{0,~1\}$, they can be regarded as classification tasks.
In addition, Logistic regression (a version of Softmax regression with $K=2$), which we have implemented in the previous notebook, was typically used to eliminate the effect of information processing by external models.
To evaluate the performance, **mutual information** $\text{MI}(y, d)$ calculating the similarity between the system's output $y$ and target $d$ is introduced, which takes the value of 1 as the maximum, and 0 as the minimum.
By summing up $\text{MI}_N$ over delay $N~(=1, 2\cdots)$, we can obtain memory capacity $C^{\text{memory}}$ and parity capacity $C^{\text{parity}}$, which was used as a measure of the information processing capacity in the paper.

In this notebook, let's implement these binary tasks and evaluate the degree of information processing of a typical *echo state network (ESN)* setup.

# 2. Purpose
Implementing **two binary tasks** typically used in reservoir computing context from scratch and understanding **how they can be applied to your setups (*e. g.,* physical ones)**.

This notebook includes implementations of
- Extension of ESN setups for classification task
- Short-term memory task
- N-bit parity check task

# 3. Preparation

In [ ]:
import sys
import math
import itertools
import numpy as np
import scipy as sp

from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/classification_task/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells
    
from utils.tqdm import trange  # progress bar
from views.memory_and_parity import *  # visualization wrapper
from tests.memory_and_parity import *  # check function

# 4. Implementation

## 4.1 Echo stete network setup for binary tasks

Here, we use a discrete ESN (DESN) as the dynamical system and evaluating $C^{\text{memory}}$ and $C^{\text{parity}}$ of the system.
The dynamics of DESN are governed by the following equation;
$$
\begin{align*}
\\
\bm{x}(t+1) &= \tanh\left(g W^\text{net} \bm{x}(t) + \bm{u}_\text{in}(s(t))\right), \\ 
\bm{u}(s) &= \begin{cases}
W^\text{in}_0 ~ (\text{s=0}) \\
W^\text{in}_1 ~ (\text{s=1})
\end{cases},
\\
\\
\end{align*}
$$
where $\tanh$ is element-wise nonlinear activation function, the constant value $g$ controls the strenth of nonlinearity, $W^\text{net} \in \mathbb{R}^{D \times D}$ is a random matrix whose spectral radius is normalized to 1 ($\rho(W^{\text{net}})=1$), and $\bm{u}: \mathbb{R}^{2} \to \mathbb{R}^D$ is random function whose weight $W^\text{in} \in \mathbb{R}^{2 \times D}$ is sampled from uniform distribution $\mathcal{U}([-\sigma_\text{in}, \sigma_\text{in}])$.

We prepare a Logistic regression model to calculate the following probabilities $y$ and predicted symbol $s_\text{out}$;
$$
\begin{align*}
\\
\bm{y}(t) &= \text{softmax}(W\bm{x}(t)+\bm{b}) ,\\
s_\text{out}(t) &= \text{argmax}_{i \in \{0,~1\}} y_i(t) ,
\\
\\
\end{align*}
$$
where $W\in \mathbb{R}^{D\times 2}$ and $\bm{b} \in \mathbb{R}^2$ are model parameters.

First, let's implement `SymbolInput` for $\bm{u}_\text{in}$.

<details>
<summary>
Hints
</summary>

- Use `self.rnd.uniform` to initialize the matrix.
- $\bm{s}(t)$ is a binary sequence, the value of which takes `0` or `1`.
- `class_num` represents the number of symbols.


In [ ]:
%load_and_run ./answer/echo_state_network.py

class SymbolInput(Module):
    def __init__(self, class_num, model_dim, bound=None, **kwargs):
        super(SymbolInput, self).__init__(**kwargs)
        self.class_num = class_num
        self.model_dim = model_dim
        if bound is None:
            bound = math.sqrt(1 / model_dim)
        # TODO initialize self.w_in
        ...

    def __call__(self, s):
        return self.predict(s)
    
    def predict(self, s):
        y = ... # TODO
        return y

In [ ]:
test_symbol_input(SymbolInput)

Next, prepare the readout module. Here, we can use the implemented class `SoftmaxRegression` (You do not need to do anything here).

In [ ]:
%load_and_run ./answer/softmax.py
%load_and_run ./answer/cross_entropy.py
%load_and_run ./answer/softmax_regression.py
%load_and_run ./answer/training.py

Sample the dynamics by generating a random symbol sequence.

The following function `emulate` samples ESN's dynamics when a symbolic sequence $s$ is given.

In [ ]:
def emulate(s, u_in, net):
    T = s.shape[0]
    record = {}
    record["s"] = s
    record["u"] = np.zeros((T, *u_in(s[0]).shape))
    record["x"] = np.zeros((T, *net.x.shape))
    for idx in range(T):
        u = ... # TODO
        ... # TODO
        record["u"][idx] = u
        record["x"][idx] = net.x
    return record

Specify the hyperparameters and observe the change in dynamics properties.

- `bound`: Value range of $W_\text{in}$ (bounded by $[-\sigma_\text{in}, \sigma_\text{in}]$)
- `esn_dim`: # of nodes $D$
- `g`: Spectral radius $g$
- `time_steps`: Length of the symbol sequence
- `class_num`: # of symbol types (set it to 2 for binary setups)

In [ ]:
bound = 1.0
esn_dim = 100
g = 0.9

time_steps = 200
class_num = 2

s = np.random.randint(0, class_num, time_steps)
u_in = SymbolInput(class_num, esn_dim, bound=bound)
net = ESN(esn_dim, g=g, a=None)

And let's check if your code is properly working by visualizing the dynamics.

In [ ]:
%matplotlib inline

net.x = np.random.randn(*net.x.shape)  # set random initial value
record = emulate(s, u_in, net)
fig = show_dynamics(record)

## 4.2 Short-term memory task


Now, it's time to implement tasks.

First, let's complete `calc_mutual_information` evaluating the task performances.

$\text{MI}(X, Y)$ between two random variables (sequences) $X$ and $Y$ can be calculated by the following equations;

$$
\text{MI}(X, Y) = P(X, Y) \log_2 \frac{P(X, Y)}{P(X) P(Y)}.
$$

<details>
<summary>
Hints
</summary>

- `X` and `Y` are binary matrices taking values of 0/1.
- Use `np.mean` for $P(X)$
- Use `np.logical_and` for calculating the joint probability $P(X, Y)$
- Add `ep` to both the fraction and numerator for avoiding overflow.
- Use `np.log2`

In [ ]:
def calc_mutual_information(X, Y, class_num, ep=1e-10):
    assert X.shape[0] == Y.shape[0]
    mi = 0.0
    for a, b in itertools.product(range(class_num), range(class_num)):
        bool_x = ... # TODO
        bool_y = ... # TODO
        p_x = ... # TODO
        p_y = ... # TODO
        p_xy = ... # TODO
        mi += ... # TODO
    return mi

In [ ]:
test_mutual_information(calc_mutual_information)

Next, implement a function `memory_task` outputting the target sequence of the **Short-memory task** for a given input sequence.

Specification of `memory_task`
- Arguments
  - Symbol sequence: $\bm{s}=\{s_{1}, \cdots, s_{L+M}\} \in \{0, 1\}^{L+M}$
  - Length: $L$
  - Delay: $N (< M)$
- Return
  - pair of sequences: $\bm{s}_\text{in}, \bm{d}_{N}$
  $$
  \begin{align*}
  \bm{s}_\text{in} &= \{u_{M}, \cdots, u_{L+M}\} \in \{0, 1\}^L \\
  \bm{d}_N &= \{u_{M-N}, \cdots, u_{L+M-N}\} \in \{0, 1\}^L \\
  \end{align*}
  $$

<details>
<summary>
Hints
</summary>

- Use slice (`a[start:stop:step]`) to realize the shift.

In [ ]:
def memory_task(s, length, delay):
    assert length > 0 and delay > 0
    s_in = ... # TODO
    d_n = ... # TODO
    return s_in, d_n

Check if it properly works (you will see a graph visualizing the memory task).

In [ ]:
%matplotlib inline

length = 40
n_max = 6
s = np.random.randint(0, 2, length + n_max)

fig = test_memory_task(memory_task, s, length, n_max)

## 4.3 N-bit parity check task

Finally, implement a function `parity_check` outputting the target sequence of the N-bit parity check task for a given input sequence.

Specification of `parity_task`
- Arguments
  - Symbol sequence: $\bm{s}=\{s_{1}, \cdots, s_{L+M}\} \in \{0, 1\}^{L+M}$
  - Length: $L$
  - Delay: $N (< M)$
- Return
  - pair of sequences: $\bm{s}_\text{in}, \bm{d}_{N}$
  $$
  \begin{align*}
  \bm{s}_\text{in} &= \{u_{M}, \cdots, u_{L+M}\} \in \{0, 1\}^L ,\\
  \bm{d}_N &= \{d_{M}, \cdots, d_{L+M}\} \in \{0, 1\}^L ,\\
  d_k &=  d_N^{\text{parity}}(k)\\
      &= \left( \sum_{i=1}^{N} u_{k-i} \right) \% 2
  \end{align*}
  $$


<details>
<summary>
Hints
</summary>

- Use `np.convolve(s, kernel, mode="valid")` ([documentation](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html)) for $\sum_{i=1}^{N} \cdots$.
- Use `%` or `np.mod` to realize modulo operation.

In [ ]:
def parity_task(s, length, delay):
    assert length > 0 and delay > 0
    s_in = ... # TODO
    d_n = ... # TODO
    d_n = d_n.astype(np.int8)
    return s_in, d_n

In [ ]:
%matplotlib inline

length = 20
n_max = 6
s = np.random.randint(0, 2, length + n_max)

fig = test_parity_task(parity_task, s, length, n_max)

# 5. Demonstration

It's time to evaluate $C^\text{memory}$ and $C^\text{parity}$.
We prepared a non-chaotic DESN with 100 nodes ($N=100, g=0.9$).
The initialization bound value of $\sigma_\text{in}$ is set to $1$.

In [ ]:
bound = 1.0
esn_dim = 200
g = 0.9

u_in = SymbolInput(2, esn_dim, bound=bound)
net = ESN(esn_dim, g=g, a=None)

Next, let's set the lengths of the washout, training, and evaluation phase (`T_washout`, `T_train`, and `T_eval`).
Note that wash-outing is necessary to get rid of the initialization effect.
Sample the training and evaluation dataset for 5,000 and 1,000-time steps, respectively.

In [ ]:
T_washout, T_train, T_eval = 100, 10000, 1000

The following function outputs the sequence of a specified task (`task_func`) performance of the system `(u_in, net)` over delays from `1` to `n_max`.
By summing up the returned value, you can get capacities ($C^\text{memory}$ and $C^\text{parity}$)
- Arguments:
  - `u_in`: `SymbolInput`
  - `net`: `ESN`
  - `task_func`: Binary task function
  - `n_max`: Integer
- Returns: `dict`
  - `"mis"`: Sequence of mutual information values
  - `"train_acc"`: Sequence of training accuracy array
  - `"train_loss"`: Sequence of training loss array
  - `"models"`: Sequence of tuned models

In [ ]:
def calc_capacity(
        u_in, net, task_func, n_max, n_epoch=10, lr=1.0, batch_size=100):
    mis = np.zeros(n_max)
    train_acc = np.zeros((n_max, n_epoch))
    train_loss = np.zeros((n_max, n_epoch))
    models = {}

    T_total = T_washout + T_train + T_eval + n_max
    s = np.random.randint(0, 2, T_total)  # generating binary sequence
    net.x = np.random.randn(*net.x.shape)  # set random initial value
    record = emulate(s, u_in, net)  # sampling dynamics

    X_raw = record["x"][T_washout:]
    X_train = X_raw[n_max:n_max + T_train]
    X_eval = X_raw[n_max + T_train:]
    S_raw = record["s"][T_washout:]
    pbar = trange(1, 1 + n_max)
    for idx, delay in enumerate(pbar):
        S, D = task_func(S_raw, T_train + T_eval, delay)
        Y = np.eye(2)[D]  # converting to one-how vector
        Y_train, D_train = Y[:T_train], D[:T_train]
        Y_eval, D_eval = Y[T_train:], D[T_train:]
        model = SoftmaxRegression(net.dim, 2)
        for epoch in range(n_epoch):
            train_acc[idx, epoch], train_loss[idx, epoch] = run_epoch(
                model, X_train, Y_train, lr=lr, train=True, batch_size=batch_size)
        Y_out = model(X_eval)
        D_out = np.argmax(Y_out, axis=1)
        mi = calc_mutual_information(D_eval, D_out, 2)
        mis[idx] = mi
        pbar.set_description(
            "[delay={:d}] {:.4f}/{:.4f}".format(delay, mi, mis[:delay].sum()))
        models[delay] = model
    return {
        "mis": mis,
        "train_acc": train_acc,
        "train_loss": train_loss,
        "models": models}


def try_optimized_model(results, delay, task_func, time_step=100):
    model = results["models"][delay]
    s = np.random.randint(0, 2, time_step + T_washout + delay)
    net.x = np.random.randn(*net.x.shape)
    record = emulate(s, u_in, net)
    S_raw = record["s"][T_washout:]
    X_eval = record["x"][T_washout + delay:]
    S_in, D_eval = task_func(S_raw, time_step, delay)
    Y_eval = np.eye(2)[D_eval]
    Y_out = model(X_eval)
    D_out = np.argmax(Y_out, axis=1)
    return show_results(delay, X_eval, S_in, D_eval, D_out, Y_eval, Y_out)

## 5.1 Short-term memory task

First, let's try the short-term memory task (`memory_task`)

In [ ]:
%matplotlib inline

res_memory = calc_capacity(u_in, net, memory_task, 50, n_epoch=10)
fig = show_capacity(res_memory)

Let's check both the good and bad cases.

In [ ]:
%matplotlib inline

fig = try_optimized_model(res_memory, 5, memory_task, 100)
fig = try_optimized_model(res_memory, 10, memory_task, 100)

## 5.2 N-bit parity check task
Next, let's try the N-bit parity check task

In [ ]:
%matplotlib inline

res_parity = calc_capacity(u_in, net, parity_task, 50, n_epoch=10)
fig = show_capacity(res_parity)

In [ ]:
%matplotlib inline

fig = try_optimized_model(res_parity, 3, parity_task, 100)
fig = try_optimized_model(res_parity, 5, parity_task, 100)

# Advanced topics

- Prepare your own physical system and assessing $C^\text{memory}$ and $C^\text{parity}$
- Design your original task function
- Implement online (real-time) sampling scripts
- Calculate information processing capacity based on discrete distribution
